In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#get shareable link from your csv file in gdrive
link = 'https://drive.google.com/open?id=1MmZS962wr2KQI2vEQbAWiEJ3exkYuso4'
_, id = link.split("=")

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('kaggle.json')  
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c grasp-and-lift-eeg-detection

 94% 143M/153M [00:01<00:00, 108MB/s] 
100% 153M/153M [00:01<00:00, 109MB/s]
  0% 0.00/5.16M [00:00<?, ?B/s]
100% 5.16M/5.16M [00:00<00:00, 84.7MB/s]
 99% 906M/915M [00:08<00:00, 115MB/s]
100% 915M/915M [00:08<00:00, 114MB/s]


In [4]:
!unzip train.zip
!unzip test.zip

Archive:  train.zip
   creating: train/
  inflating: train/subj10_series1_data.csv  
  inflating: train/subj10_series1_events.csv  
  inflating: train/subj10_series2_data.csv  
  inflating: train/subj10_series2_events.csv  
  inflating: train/subj10_series3_data.csv  
  inflating: train/subj10_series3_events.csv  
  inflating: train/subj10_series4_data.csv  
  inflating: train/subj10_series4_events.csv  
  inflating: train/subj10_series5_data.csv  
  inflating: train/subj10_series5_events.csv  
  inflating: train/subj10_series6_data.csv  
  inflating: train/subj10_series6_events.csv  
  inflating: train/subj10_series7_data.csv  
  inflating: train/subj10_series7_events.csv  
  inflating: train/subj10_series8_data.csv  
  inflating: train/subj10_series8_events.csv  
  inflating: train/subj11_series1_data.csv  
  inflating: train/subj11_series1_events.csv  
  inflating: train/subj11_series2_data.csv  
  inflating: train/subj11_series2_events.csv  
  inflating: train/subj11_series3_data.c

In [0]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import random
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score as auc
from scipy.interpolate import BSpline


In [0]:
USE_CUDA = 1

In [0]:
labels = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [0]:
def prepare_data_train(fname):
    """ read and prepare training data """
    data = pd.read_csv(fname)
    events_fname = fname.replace('_data','_events')
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels


scaler= StandardScaler()
def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    
    return X_prep
def data_preprocess_test(X):
    X_prep=scaler.transform(X)
    
    return X_prep

In [0]:
subjects=range(1,13)

In [0]:
def load_data(subjects,series):
  y_raw= []
  raw = []
  for subject in subjects:
    print("Loading from subject ",subject)
    
    for ser in series:
      fname ='train/subj%d_series%d_data.csv' % (subject,ser)
      # print(fname)
      data,labels=prepare_data_train(fname)
      raw.append(data)
      y_raw.append(labels)

    X = pd.concat(raw)
    y = pd.concat(y_raw)
      
    X =np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))
  return X,y


In [11]:
X_train,y_train=load_data(subjects,series =[1,2,5,6,7,8])
X_train=data_preprocess_train(X_train)
print(X_train.shape,y_train.shape)

Loading from subject  1
Loading from subject  2
Loading from subject  3
Loading from subject  4
Loading from subject  5
Loading from subject  6
Loading from subject  7
Loading from subject  8
Loading from subject  9
Loading from subject  10
Loading from subject  11
Loading from subject  12
(13239043, 32) (13239043, 6)


In [12]:
X_test,y_test=load_data(subjects,series=[3,4])
X_test=data_preprocess_test(X_test)
print(X_test.shape,y_test.shape)


Loading from subject  1
Loading from subject  2
Loading from subject  3
Loading from subject  4
Loading from subject  5
Loading from subject  6
Loading from subject  7
Loading from subject  8
Loading from subject  9
Loading from subject  10
Loading from subject  11
Loading from subject  12
(4746711, 32) (4746711, 6)


In [0]:
# some parameteres for the model
num_features = 32
window_size = 1024
batch_size=2000

In [0]:
def get_batch(dataset,target, batch_size=2000, val=False, index=None):
    if val == False:
        index = random.randint(window_size, len(dataset) - 16 * batch_size)
        indexes = np.arange(index, index + 16*batch_size, 16)

    else:
        indexes = np.arange(index, index + batch_size)
    
    batch = np.zeros((batch_size, num_features, window_size//4))
    
    b = 0
    for i in indexes:
        start = i - window_size if i - window_size > 0 else 0
        
        tmp = dataset[start:i]
        batch[b,:,:] = tmp[::4].transpose()        
        b += 1

    targets = target[indexes]
    return torch.DoubleTensor(batch), torch.DoubleTensor(targets) 

In [0]:
class ConvLayer(nn.Module):
  def __init__(self, in_channels=32, out_channels=64, kernel_size=9):
    super(ConvLayer, self).__init__()
    self.conv = nn.Conv1d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=1)
    
  def forward(self, x):
    return F.relu(self.conv(x))

In [0]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=6, in_channels=64, out_channels=32 , kernel_size=9, num_routes=32 * 6 ):
        super(PrimaryCaps, self).__init__()
        self.num_routes = num_routes
        self.capsules = nn.ModuleList([
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
            for _ in range(num_capsules)])

    def forward(self, x):
        
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0),self.num_routes,-1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        return squared_norm



In [0]:
class DigitCaps(nn.Module):
    def __init__(self, num_capsules=6, num_routes=32 * 6 , in_channels=1, out_channels=16):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)
        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if USE_CUDA:
            b_ij = b_ij.cuda()
        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij, dim=1)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor



    

In [0]:
class Decoder(nn.Module):
    def __init__(self, input_width=32, input_height=256):
        super(Decoder, self).__init__()
        self.input_width = input_width
        self.input_height = input_height
      
        self.reconstraction_layers = nn.Sequential(
            nn.Linear(16 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 8192),
            nn.ReLU(inplace=True),
            nn.Linear(8192,self.input_width*self.input_height),
            nn.Sigmoid()
        )

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))

        classes = F.softmax(classes, dim=0)
       
        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(6))
        if USE_CUDA:
            masked = masked.cuda()
        masked = masked.index_select(dim=0, index=Variable(max_length_indices.squeeze(1).data))

        t = (x * masked[:, :, None, None]).view(x.size(0), -1)
        reconstructions = self.reconstraction_layers(t)
        reconstructions = reconstructions.view(-1,self.input_width,self.input_height)
        return reconstructions,masked  




In [0]:
class CapsNet(nn.Module):
    def __init__(self, config=None):
        super(CapsNet, self).__init__()
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()
        self.decoder = Decoder()
        self.mse_loss = nn.MSELoss()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
       
        reconstructions, masked = self.decoder(output, data)
        return output, reconstructions, masked

    def loss(self, data, x, target, reconstructions):
        return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    def margin_loss(self, x, labels, size_average=True):
        batch_size = x.size(0)

        v_c = torch.sqrt((x ** 2).sum(dim=2, keepdim=True))
        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)

        loss = labels * left + 0.5 * (1.0 - labels) * right
        loss = loss.sum(dim=1).mean()

        return loss

    def reconstruction_loss(self, data, reconstructions):
        loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
        return loss * 0.0005

In [0]:
capsule_net = CapsNet().double()
if USE_CUDA == 1:
    capsule_net = capsule_net.cuda()
optim = torch.optim.Adam(capsule_net.parameters())#,lr=1e-3, eps=1e-10)

In [21]:
from torch.nn.modules.module import _addindent
def torch_summarize(model, show_weights=True, show_parameters=True):
    """Summarizes torch model by showing trainable parameters and weights."""
    tmpstr = model.__class__.__name__ + ' (\n'
    for key, module in model._modules.items():
        # if it contains layers let call it recursively to get params and weights
        if type(module) in [
            torch.nn.modules.container.Container,
            torch.nn.modules.container.Sequential
        ]:
            modstr = torch_summarize(module)
        else:
            modstr = module.__repr__()
        modstr = _addindent(modstr, 2)

        params = sum([np.prod(p.size()) for p in module.parameters()])
        weights = tuple([tuple(p.size()) for p in module.parameters()])

        tmpstr += '  (' + key + '): ' + modstr 
        if show_weights:
            tmpstr += ', weights={}'.format(weights)
        if show_parameters:
            tmpstr +=  ', parameters={}'.format(params)
        tmpstr += '\n'   

    tmpstr = tmpstr + ')'
    return tmpstr

print(torch_summarize(capsule_net))

CapsNet (
  (conv_layer): ConvLayer(
    (conv): Conv1d(32, 64, kernel_size=(9,), stride=(1,))
  ), weights=((64, 32, 9), (64,)), parameters=18496
  (primary_capsules): PrimaryCaps(
    (capsules): ModuleList(
      (0): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
      (1): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
      (2): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
      (3): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
      (4): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
      (5): Conv1d(64, 32, kernel_size=(9,), stride=(2,))
    )
  ), weights=((32, 64, 9), (32,), (32, 64, 9), (32,), (32, 64, 9), (32,), (32, 64, 9), (32,), (32, 64, 9), (32,), (32, 64, 9), (32,)), parameters=110784
  (digit_capsules): DigitCaps(), weights=((1, 192, 6, 16, 1),), parameters=18432
  (decoder): Decoder(
    (reconstraction_layers): Sequential(
      (0): Linear(in_features=96, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=8192

In [0]:
bs = batch_size
def train(traindata,y_train, epochs, printevery=100, shuffle=True):
    capsule_net.train()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(traindata)//bs):
            optim.zero_grad()
            x, y = get_batch(traindata,y_train,batch_size=batch_size)
            if USE_CUDA == 1:
                data = Variable(x).cuda()
                y = Variable(y).cuda()
            
            output, reconstructions, masked = capsule_net(data)
            loss = capsule_net.loss(data, output, y, reconstructions)

            loss.backward()
            optim.step()
            train_loss = loss.item()
            total_loss += train_loss
            
            if (i + 1) % printevery == 0:
                print("epoch: %d, iter %d/%d, loss %.4f"%(epoch + 1, i + 1, len(traindata)//2000, total_loss/printevery))
                total_loss = 0
                

In [0]:
train(X_train,y_train,1)

epoch: 1, iter 100/6619, loss 10.9304


KeyboardInterrupt: ignored

In [0]:
!mkdir models
torch.save(capsule_net.state_dict(), "./models/CapsNet")

mkdir: cannot create directory ‘models’: File exists


In [0]:
def getPredictions(data,y_test):
    capsule_net.eval()
    p = []
    res = []
    i = window_size
    bs = 2000
    while i < len(data):
        if i + bs > len(data):
            bs = len(data) - i
        x, y = get_batch(data,y_test, bs, index=i, val=True)
        x = (x)
        x = x.cuda()
        y = y
        output, reconstructions, masked = capsule_net(x)
        preds=masked
        preds = preds.squeeze(1)
        p.append(np.array(preds.cpu().data))
        res.append(np.array(y.data))
        i += bs
    preds = p[0]
    for i in p[1:]:
        preds = np.vstack((preds,i))
    targs = res[0]
    for i in res[1:]:
        targs = np.vstack((targs, i))
    return preds, targs



In [0]:
val_preds, val_targs=getPredictions(X_test,y_test)

In [0]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [0]:
accuracy=[]
for j in range(6):
  accuracy.append(accuracy_score(val_targs[:, j],val_preds[:, j]))
print("accurecy score ",np.mean(accuracy))

accurecy score  0.8189604933209179


In [0]:
for j in range(6):
  print(confusion_matrix(val_targs[:, j],val_preds[:, j]))

[[3649576  980911]
 [  91010   24190]]
[[4334656  295831]
 [ 109114    6086]]
[[2807135 1823352]
 [  52349   62851]]
[[4137211  493276]
 [  96365   18835]]
[[4582266   48221]
 [ 114114    1086]]
[[3667364  963123]
 [  87275   27925]]
